This script gathers price data from Aave API
https://aave-api-v2.aave.com/

In [ ]:
pip install requests

In [ ]:
import requests
import pandas as pd
import datetime
from datetime import timedelta
import time

In [ ]:
def isnan(string):
    return string == 'NaN'

## V1

In [ ]:
pool = '0x24a42fd28c976a61df5d00d0599c34c4f90748c8' #v1
date = datetime.datetime(2020, 12, 31)
date_string = date.strftime('%m-%d-%Y')
lst = []

In [ ]:
def process_aave_v1(raw):
    for i in range(0,len(raw)):
        temp = []
        if raw[i]['symbol'] == 'USDC':
            if isnan(raw[i]['referenceItem']['liquidityRate']): #if item is empty, pool does not exists
                temp.append(['empty', 0, 0, 0, 0, 0, 0])
            else:
                epoch = int(raw[i]['referenceItem']['timestamp'])
                time = datetime.datetime.utcfromtimestamp(epoch)
                supply = float(raw[i]['referenceItem']['liquidityRate'])
                borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
                supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
                borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
                temp.append(['usdc', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
                
        elif raw[i]['symbol'] == 'USDT':
            if isnan(raw[i]['referenceItem']['liquidityRate']): #if item is empty, pool does not exists
                temp.append(['empty', 0, 0, 0, 0, 0, 0])
            else:
                epoch = int(raw[i]['referenceItem']['timestamp'])
                time = datetime.datetime.utcfromtimestamp(epoch)
                supply = float(raw[i]['referenceItem']['liquidityRate'])
                borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
                supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
                borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
                temp.append(['usdt', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
                
        elif raw[i]['symbol'] == 'DAI':
            if isnan(raw[i]['referenceItem']['liquidityRate']): #if item is empty, pool does not exists
                temp.append(['empty', 0, 0, 0, 0, 0, 0])
            else:
                epoch = int(raw[i]['referenceItem']['timestamp'])
                time = datetime.datetime.utcfromtimestamp(epoch)
                supply = float(raw[i]['referenceItem']['liquidityRate'])
                borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
                supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
                borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
                temp.append(['dai', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
                
        elif raw[i]['symbol'] == 'ETH':
            if isnan(raw[i]['referenceItem']['liquidityRate']): #if item is empty, pool does not exists
                temp.append(['empty', 0, 0, 0, 0, 0, 0])
            else:
                epoch = int(raw[i]['referenceItem']['timestamp'])
                time = datetime.datetime.utcfromtimestamp(epoch)
                supply = float(raw[i]['referenceItem']['liquidityRate'])
                borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
                supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
                borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
                temp.append(['weth', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
                
        elif raw[i]['symbol'] == 'WBTC':
            if isnan(raw[i]['referenceItem']['liquidityRate']): #if item is empty, pool does not exists
                temp.append(['empty', 0, 0, 0, 0, 0, 0])
            else:
                epoch = int(raw[i]['referenceItem']['timestamp'])
                time = datetime.datetime.utcfromtimestamp(epoch)
                supply = float(raw[i]['referenceItem']['liquidityRate'])
                borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
                supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
                borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
                temp.append(['wbtc', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
                
        elif raw[i]['symbol'] == 'UNI':
            if isnan(raw[i]['referenceItem']['liquidityRate']): #if item is empty, pool does not exists
                temp.append(['empty', 0, 0, 0, 0, 0, 0])
            else:
                epoch = int(raw[i]['referenceItem']['timestamp'])
                time = datetime.datetime.utcfromtimestamp(epoch)
                supply = float(raw[i]['referenceItem']['liquidityRate'])
                borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
                supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
                borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
                temp.append(['uni', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
                
        elif raw[i]['symbol'] == 'LINK':
            if isnan(raw[i]['referenceItem']['liquidityRate']): #if item is empty, pool does not exists
                temp.append(['empty', 0, 0, 0, 0, 0, 0])
            else:
                epoch = int(raw[i]['referenceItem']['timestamp'])
                time = datetime.datetime.utcfromtimestamp(epoch)
                supply = float(raw[i]['referenceItem']['liquidityRate'])
                borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
                supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
                borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
                temp.append(['link', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
                
        else:
            continue

        lst.append(temp)

            
       

In [ ]:
cont = True

while cont:
    query = f'https://aave-api-v2.aave.com/data/liquidity/v1?poolId={pool}&date={date_string}'
    response = requests.get(query)
    raw = response.json()
    process_aave_v1(raw)
    date = (date - timedelta(days=1))
    date_string = date.strftime('%m-%d-%Y')
    print(date_string)
    time.sleep(3)


In [ ]:
uni = [item for sublist in lst[0::7] for item in sublist]
wbtc = [item for sublist in lst[1::7] for item in sublist]
link = [item for sublist in lst[2::7] for item in sublist]
dai = [item for sublist in lst[3::7] for item in sublist]
usdc = [item for sublist in lst[4::7] for item in sublist]
usdt = [item for sublist in lst[5::7] for item in sublist]
weth = [item for sublist in lst[6::7] for item in sublist]

final_lst = [uni, wbtc, link, dai, usdc, usdt, weth]

In [ ]:
currencies = ['uni', 'wbtc', 'link', 'dai', 'usdc', 'usdt', 'weth']
for i in range(0, len(currencies)):
    df_aave = pd.DataFrame(final_lst[i], columns=['asset', 'epoch', 'timestamp', 'supply', 'borrow', 'supply_ma30', 'borrow_ma30'])
    df = df_aave.copy()
    #df = df.sort_values(by='timestamp', ascending=True)
    df = df.reset_index(drop=True)
    #store in csv
    df.to_csv(f'data/aave_{currencies[i]}_interest_v1.csv', index=False)


## V2

In [ ]:
pool = '0xb53c1a33016b2dc2ff3653530bff1848a515c8c5' #v2
date = datetime.datetime(2022, 11, 13)
date_string = date.strftime('%m-%d-%Y')

In [ ]:
lst = []

In [ ]:
def process_aave_v2(raw):
    for i in range(0,len(raw)):
        temp = []
        if raw[i]['symbol'] == 'USDC':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            supply = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
            borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
            temp.append(['usdc', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
        elif raw[i]['symbol'] == 'USDT':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            supply = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
            borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
            temp.append(['usdt', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
        elif raw[i]['symbol'] == 'DAI':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            supply = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
            borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
            temp.append(['dai', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
        elif raw[i]['symbol'] == 'WETH':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            supply = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
            borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
            temp.append(['weth', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
        elif raw[i]['symbol'] == 'WBTC':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            supply = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
            borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
            temp.append(['wbtc', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
        elif raw[i]['symbol'] == 'UNI':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            supply = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
            borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
            temp.append(['uni', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
        elif raw[i]['symbol'] == 'LINK':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            supply = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
            borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
            temp.append(['link', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
        else:
            continue
            
        lst.append(temp)

In [ ]:
cont = True

while cont:
    query = f'https://aave-api-v2.aave.com/data/liquidity/v2?poolId={pool}&date={date_string}'
    response = requests.get(query)
    raw = response.json()
    process_aave_v2(raw)
    date = (date - timedelta(days=1))
    date_string = date.strftime('%m-%d-%Y')
    print(date_string)
    time.sleep(3)


In [ ]:
uni = [item for sublist in lst[0::7] for item in sublist]
wbtc = [item for sublist in lst[1::7] for item in sublist]
link = [item for sublist in lst[2::7] for item in sublist]
dai = [item for sublist in lst[3::7] for item in sublist]
usdc = [item for sublist in lst[4::7] for item in sublist]
weth = [item for sublist in lst[5::7] for item in sublist]
usdt = [item for sublist in lst[6::7] for item in sublist]

final_lst = [uni, wbtc, link, dai, usdc, weth, usdt]

In [ ]:
currencies = ['uni', 'wbtc', 'link', 'dai', 'usdc', 'weth', 'usdt']
for i in range(0, len(currencies)):
    df_aave = pd.DataFrame(final_lst[i], columns=['asset', 'epoch', 'timestamp', 'supply', 'borrow', 'supply_ma30', 'borrow_ma30'])
    df = df_aave.copy()
    df = df.sort_values(by='timestamp', ascending=True)
    df = df.reset_index(drop=True)
    #store in csv
    df.to_csv(f'data/aave_{currencies[i]}_interest_v2.csv', index=False)
